# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 championship. He drove for Red Bull Racing and clinched his first ever World Drivers' Championship that season.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are an expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [9]:
context_user = [
    {'role':'system', 'content':"""You are an expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are going to answer the question of the user giving the name of the rider and
    the name of the team, following the format:
    Drive:
    Team:

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# **Chatbot No. 1 Role: Expert in F1**

In [12]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Validate API Key
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY not found. Check your .env file.")

In [13]:
def get_completion(user_message, model="gpt-3.5-turbo", temperature=0): 
    client = OpenAI(api_key=OPENAI_API_KEY)  # Initialize client inside function
    messages = [{"role": "user", "content": user_message}]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature, 
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in API call: {e}")
        return "An error occurred. Please try again."

In [24]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(api_key=OPENAI_API_KEY)  # Initialize client
    newcontext = context.copy()
    newcontext.append({'role': 'user', 'content': "question: " + user_message})

    # Limit context size to avoid token overflow
    if len(newcontext) > 20:
        newcontext = newcontext[-20:]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in API call: {e}")
        return "An error occurred. Please try again."

In [15]:
# Ensure context is initialized globally
context = [
    {"role": "system", "content": """You are an expert in F1.
    You are going to answer the question of the user giving the rider details such name, his Country, team name, 
    how many points earned within championship asked, how many championships won thru his career and including the years won.
    You will respond using the following the format:
    The winner of "year of the question" F1 Championship is
    Driver:
    Country:
    Team:
    Points:
    Total of Championships:
    Championship Years:

     Who won the 2010 f1 championship?
     The winner of 2010 F1 Championship is
     Driver: Sebastian Vettel.
     Country: Germany.
     Team: Red Bull Renault.
     Points: 256
     Total of Championships won: 4
     Championship Years: 2010, 2011, 2012, 2013

     Who won the 2017 f1 championship?
     The winner of 2017 F1 Championship is
     Driver: Lewis Hamilton.
     Country: United Kingdom.
     Team: Mercedes.
     Points: 363
     Total of Championships won: 7
     Championship Years: 2008, 2014, 2015, 2017, 2018, 2019, 2020"""}
]

panels = []  # Initialize an empty list for displaying messages

def collect_messages(_):
    global context, panels
    user_message = inp.value.strip()  # Capture user input
    if not user_message:  # Ignore empty input
        return
    inp.value = ''  # Clear the input field

    context.append({'role': 'user', 'content': user_message})  # Add user message to the context
    try:
        response = return_OAIResponse(user_message, context)  # Call the model and get a response
        context.append({'role': 'assistant', 'content': response})  # Add assistant response to the context
    except Exception as e:
        response = f"An error occurred: {e}"

    # Update panels with reversed order (latest message at the top)
    panels.insert(0, pn.Row('User:', pn.pane.Markdown(user_message, width=1000, styles={'color': '#FFFFFF'})))  # User message
    panels.insert(0, pn.Row('Assistant:', pn.pane.Markdown(response, width=1000, styles={'color': '#FFD700', 'background-color': '#333333'})))  # Assistant response

    return pn.Column(*panels)

In [16]:
import panel as pn  # GUI
pn.extension()

# TextInput and Button for user interaction
inp = pn.widgets.TextInput(value="", placeholder='Enter your F1 question here…')
button_conversation = pn.widgets.Button(name="Ask!")

# Bind the button to the collect_messages function
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard with custom styles and scroll enabled
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300, scroll=True, styles={
        'background-color': '#1E1E1E',
        'overflow-y': 'auto'  # To ensure a vertical scroll
    }),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'b8572082-aa5a-45eb-90f9-f8cda30f55ac': {'version…

# **Chatbot No. 2 Role: Expert in FIFA, international football competitions**

In [30]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [31]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(api_key=OPENAI_API_KEY)  # Initialize client
    newcontext = context.copy()
    newcontext.append({'role': 'user', 'content': "question: " + user_message})

    # Limit context size to avoid token overflow
    if len(newcontext) > 20:
        newcontext = newcontext[-20:]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in API call: {e}")
        return "An error occurred. Please try again."

In [32]:
#pip install datetime

In [33]:
# Ensure context is initialized globally
context = [
    {"role": "system", "content": """You are an expert in FIFA and International Football Competitions.
    You can answer questions about the last international competitions won by a specific player. \
    In addition you can provide detailed information about the winner of a specific competition. \
    Your responses must strictly adhere to the following guidelines: \
    1. You MUST ALLWAYS follow the responses format examples provided on below section named: QUESTIONS EXAMPLES AND RESPONSE FORMAT. \
    2. ONLY WHEN a question is related to dates after September 2021 you MUST answer using the following DISCLAIMER: \
    "Note: My knowledge cutoff date is September 2021. I cannot provide information on events or competitions after this date. \
    But, I can provide the last information available and related to the question." \
    3. However, if the user provides relevant and recent data, you may process it and generate an answer using the provided context. \
    Ensure your responses are structured and consistent with the examples provided below. \
    Your responses should NEVER omit any of the required fields.
    
    QUESTIONS EXAMPLES AND RESPONSE FORMAT:\
    - If asked about the last international competition won by a specific player:\
        RESPONSE FORMAT:
        The last international competition won by [Player Name] was [Competition Name] in [Year]. [Player Name] scored a total of [# Goals] goals and [# Assists] assists.\

    - If asked about a specific competition winner:\
        RESPONSE FORMAT:
        Competition: [Competition Name]
        Year: [Year]
        Winning Team: [Team Name]
        Final Result: [Score, e.g., 4-2 against Croatia]
        Winning Team Top Scorer(s): [Player(s) Name] scored [# Goals] goals in this competition. If there are a draw in total goals, include all players of the team with eaqual total goals
        Total [Competition Name] Championships Won: [# Titles].
    """}
]

# Add examples using context.extend for Few-Shots Learning
context.extend([
    # Example for Question Style 1
    {"role": "user", "content": "What was the last international competition won by Lionel Messi?"},
    {"role": "assistant", "content": "The last international competition won by Lionel Messi was the FIFA World Cup in 2022. Messi scored a total of 7 goals and provided 3 assists."},

    {"role": "user", "content": "What was the last international competition won by Cristiano Ronaldo?"},
    {"role": "assistant", "content": "The last international competition won by Cristiano Ronaldo was the UEFA Nations League in 2019. Ronaldo scored a total of 3 goals and provided 1 assist."},

    # Example for Question Style 2
    {"role": "user", "content": "Which team won the 2018 FIFA World Cup?"},
    {"role": "assistant", "content": '''
     Competition: FIFA World Cup
     Year: 2018
     Winning Team: France
     Final Result: 4-2 against Croatia
     Winning Team Top Scorer(s): Antoine Griezmann and Kylian Mbappé scored 4 goals in this competition.
     Total Championships Won: 2 FIFA World Cups by France.'''},

    {"role": "user", "content": "Who won the 2021 Copa America?"},
    {"role": "assistant", "content": '''
     Competition: Copa America
     Year: 2021
     Winning Team: Argentina
     Final Result: 1-0 against Brazil
     Winning Team Top Scorer(s): Lionel Messi scored 4 goals in this competition.
     Total Championships Won: 15 Copa America titles by Argentina.'''},

    {"role": "user", "content": "Which team won the 2010 FIFA World Cup?"},
    {"role": "assistant", "content": '''
     Competition: FIFA World Cup
     Year: 2010
     Winning Team: Spain
     Final Result: 1-0 against the Netherlands
     Winning Team Top Scorer(s): David Villa scored 5 goals in this competition.
     Total Championships Won: 1 FIFA World Cup by Spain.'''}
])

panels = []  # Initialize an empty list for displaying messages

def collect_messages(_):
    global context, panels
    user_message = inp.value.strip()  # Capture user input
    if not user_message:  # Ignore empty input
        return
    inp.value = ''  # Clear the input field

    context.append({'role': 'user', 'content': user_message})  # Add user message to the context
    try:
        response = return_OAIResponse(user_message, context)  # Call the model and get a response
        context.append({'role': 'assistant', 'content': response})  # Add assistant response to the context
    except Exception as e:
        response = f"An error occurred: {e}"

    # Update panels with reversed order (latest message at the top)
    panels.insert(0, pn.Row('User:', pn.pane.Markdown(user_message, width=1000, styles={'color': '#FFFFFF'})))  # User message
    panels.insert(0, pn.Row('Assistant:', pn.pane.Markdown(response, width=1000, styles={'color': '#FFD700', 'background-color': '#333333'})))  # Assistant response

    return pn.Column(*panels)

Codes tested but no results improvement was noticed.

#1

def collect_messages(_):
    global context, panels
    user_message = inp.value.strip()  # Capture user input
    if not user_message:  # Ignore empty input
        return
    inp.value = ''  # Clear the input field

    # Extract year from user message
    year = None
    for word in user_message.split():
        if word.isdigit() and len(word) == 4:  # Check if it's a 4-digit year
            year = int(word)
            break

    current_year = datetime.now().year

    # Add a disclaimer if the question involves a date beyond the cutoff
    disclaimer = "Note: My knowledge cutoff date is September 2021. I cannot provide information on events or competitions beyond this date."
    if year and year > 2021:
        response = f"{disclaimer} I can, however, provide information up to 2021 or process recent data if you provide it."
    else:
        # Add user message to the context
        context.append({'role': 'user', 'content': user_message})
        try:
            # Call the model and get a response
            response = return_OAIResponse(user_message, context)
            context.append({'role': 'assistant', 'content': response})
        except Exception as e:
            response = f"An error occurred: {e}"

    # Update the Panel display
    panels.insert(0, pn.Row('User:', pn.pane.Markdown(user_message, width=1000, styles={'color': '#FFFFFF'})))  # User message
    panels.insert(0, pn.Row('Assistant:', pn.pane.Markdown(response, width=1000, styles={'color': '#FFD700', 'background-color': '#333333'})))  # Assistant response

#2

def classify_question(user_message):
    """
    Classify the type of question based on the user input.
    Returns either 'player' or 'competition'.
    """
    if "last international competition won" in user_message.lower():
        return "player"
    elif "team won" in user_message.lower() or "competition" in user_message.lower():
        return "competition"
    return "unknown"  # Default classification for other cases

def validate_response_format(response, question_type):
    """
    Validate the format of the response based on the question type.
    Returns True if the response is valid, False otherwise.
    """
    if question_type == "competition":
        required_fields = [
            "Competition:", 
            "Year:", 
            "Winning Team:", 
            "Final Result:", 
            "Winning Team Top Scorer(s):", 
            "Total Championships Won:"
        ]
    elif question_type == "player":
        required_fields = [
            "The last international competition won by", 
            "was", 
            "scored a total of", 
            "goals and", 
            "assists."
        ]
    else:
        return True  # If the type is unknown, skip validation

    # Check if all required fields are in the response
    for field in required_fields:
        if field not in response:
            return False
    return True

def collect_messages(_):
    global context, panels
    user_message = inp.value.strip()  # Capture user input
    if not user_message:  # Ignore empty input
        return
    inp.value = ''  # Clear the input field

    # Add user message to the context
    context.append({'role': 'user', 'content': user_message})

    # Classify the question type
    question_type = classify_question(user_message)

    try:
        # Call the model and get a response
        response = return_OAIResponse(user_message, context)

        # Validate the response format
        if not validate_response_format(response, question_type):
            response = (
                "The response format was incorrect or incomplete. Please ensure the query matches "
                "the required format for competition or player-related questions."
            )

        # Add assistant response to the context
        context.append({'role': 'assistant', 'content': response})

    except Exception as e:
        response = f"An error occurred: {e}"

    # Update the Panel display
    panels.insert(0, pn.Row('User:', pn.pane.Markdown(user_message, width=1000, styles={'color': '#FFFFFF'})))  # User message
    panels.insert(0, pn.Row('Assistant:', pn.pane.Markdown(response, width=1000, styles={'color': '#FFD700', 'background-color': '#333333'})))  # Assistant response

    return pn.Column(*panels) 

In [34]:
import panel as pn  # GUI
pn.extension()

# TextInput and Button for user interaction
inp = pn.widgets.TextInput(value="", placeholder='Enter your FIFA/International Football Chammpionship question here…')
button_conversation = pn.widgets.Button(name="Ask!")

# Bind the button to the collect_messages function
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard with custom styles and scroll enabled
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300, scroll=True, styles={
        'background-color': '#1E1E1E',
        'overflow-y': 'auto'  # To ensure a vertical scroll
    }),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'fb7c1987-4bd6-4320-ab2c-c7cb055e8bb7': {'version…

# **Chatbot No. 3 Role: Expert Movies Reviews**

In [48]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [49]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(api_key=OPENAI_API_KEY)  # Initialize client
    newcontext = context.copy()
    newcontext.append({'role': 'user', 'content': "question: " + user_message})

    # Limit context size to avoid token overflow
    if len(newcontext) > 20:
        newcontext = newcontext[-20:]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in API call: {e}")
        return "An error occurred. Please try again."

In [50]:
# Ensure context is initialized globally
context = [
    {"role": "system", "content": """"You are an expert in movies, including awards, release dates, directors, cast, and ratings. \
     You will provide detailed answers about a movie's awards, release date, director, main actors, and ratings from IMDb and Rotten Tomatoes. \
     Ensure your responses follow the specified format." \
    Your responses must strictly adhere to the following guidelines: \
    1. You MUST ALLWAYS follow the responses format examples provided on below section named: QUESTIONS EXAMPLES AND RESPONSE FORMAT. \
    2. ONLY WHEN a question is related to dates after September 2021 you MUST answer using the following DISCLAIMER: \
    "Note: My knowledge cutoff date is September 2021. I cannot provide information after this date. \
    But, I can provide the last information available and related to the question." \
    
    
    QUESTIONS EXAMPLES AND RESPONSE FORMAT:\
    - For all questions, the chatbot will respond with the following format:\
        RESPONSE FORMAT:
        Movie: [Name of the Movie]
        Release Date: [Month Day, Year]
        Awards Won: [List of Awards Won]
        Director: [Name of the Director]
        Main Cast: [List of Main Actors]
        IMDb Rating: [IMDb Rating]
        Rotten Tomatoes Rating: [Rotten Tomatoes Rating]
    """}
]

# Add examples using context.extend for Few-Shots Learning
context.extend([
    # Example for Question Style 1
    {"role": "user", "content": "What awards did Titanic win?"},
    {"role": "assistant", "content": 
     """Movie: Titanic
        Release Date: December 19, 1997
        Awards Won: 11 Academy Awards, including Best Picture, Best Director, and Best Original Song
        Director: James Cameron
        Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane
        IMDb Rating: 7.9/10
        Rotten Tomatoes Rating: 88%"""},

    {"role": "user", "content": "What awards did Parasite win?"},
    {"role": "assistant", "content": 
     """Movie: Parasite
        Release Date: May 30, 2019
        Awards Won: 4 Academy Awards, including Best Picture, Best Director, Best Original Screenplay, and Best International Feature Film
        Director: Bong Joon-ho
        Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong
        IMDb Rating: 8.5/10
        Rotten Tomatoes Rating: 99%"""},

    {"role": "user", "content": "What awards did The Godfather win?"},
    {"role": "assistant", "content": 
     """Movie: The Godfather
        Release Date: March 24, 1972
        Awards Won: 3 Academy Awards, including Best Picture, Best Actor (Marlon Brando), and Best Adapted Screenplay
        Director: Francis Ford Coppola
        Main Cast: Marlon Brando, Al Pacino, James Caan
        IMDb Rating: 9.2/10
        Rotten Tomatoes Rating: 97%"""},

    # Example for Question Style 2
    {"role": "user", "content": "What was the last movie award won by Alfonso Cuarón?"},
    {"role": "assistant", "content": 
     """Last award won by Alfonso Cuarón was due to the movie title "Roma"
        Movie: Roma
        Release Date: November 21, 2018
        Awards Won: Academy Award for Best Director, Academy Award for Best Cinematography, \
                    Academy Award for Best Foreign Language Film, Golden Globe for Best Director, \
                    BAFTA for Best Film Not in the English Language, and more.
        Director: Alfonso Cuarón
        Main Cast: Yalitza Aparicio, Marina de Tavira, Diego Cortina Autrey
        IMDb Rating: 7.7/10
        Rotten Tomatoes Rating: 96%"""},

    {"role": "user", "content": "What was the last movie award won by Guillermo del Toro?"},
    {"role": "assistant", "content": 
     """Last award won by Guillermo del Toro was due to the movie title "The Shape of Water"
        Movie: The Shape of Water
        Release Date: December 1, 2017
        Awards Won: Academy Award for Best Picture, Academy Award for Best Director, \
                    Academy Award for Best Original Score, Academy Award for Best Production Design, \
                    Golden Globe for Best Director, BAFTA for Best Director
        Director: Guillermo del Toro
        Main Cast: Sally Hawkins, Michael Shannon, Richard Jenkins, Octavia Spencer, Doug Jones
        IMDb Rating: 7.3/10
        Rotten Tomatoes Rating: 92%"""},

    {"role": "user", "content": "What was the last movie award won by Clint Eastwood?"},
    {"role": "assistant", "content": 
     """Last award won by Clint Eastwood was due to the movie title "Unforgiven"
        Movie: Unforgiven
        Release Date: August 7, 1992
        Awards Won: Academy Award for Best Picture, Academy Award for Best Director, \
                    Academy Award for Best Supporting Actor (Gene Hackman), \
                    Academy Award for Best Film Editing
        Director: Clint Eastwood
        Main Cast: Clint Eastwood, Gene Hackman, Morgan Freeman, Richard Harris
        IMDb Rating: 8.2/10
        Rotten Tomatoes Rating: 96%"""}
])

panels = []  # Initialize an empty list for displaying messages

def collect_messages(_):
    global context, panels
    user_message = inp.value.strip()  # Capture user input
    if not user_message:  # Ignore empty input
        return
    inp.value = ''  # Clear the input field

    context.append({'role': 'user', 'content': user_message})  # Add user message to the context
    try:
        response = return_OAIResponse(user_message, context)  # Call the model and get a response
        context.append({'role': 'assistant', 'content': response})  # Add assistant response to the context
    except Exception as e:
        response = f"An error occurred: {e}"

    # Update panels with reversed order (latest message at the top)
    panels.insert(0, pn.Row('User:', pn.pane.Markdown(user_message, width=1000, styles={'color': '#FFFFFF'})))  # User message
    panels.insert(0, pn.Row('Assistant:', pn.pane.Markdown(response, width=1000, styles={'color': '#FFD700', 'background-color': '#333333'})))  # Assistant response

    return pn.Column(*panels)

In [51]:
import panel as pn  # GUI
pn.extension()

# TextInput and Button for user interaction
inp = pn.widgets.TextInput(value="", placeholder='Enter your movie awards question here…')
button_conversation = pn.widgets.Button(name="Ask!")

# Bind the button to the collect_messages function
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard with custom styles and scroll enabled
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1000, scroll=True, styles={
        'background-color': '#1E1E1E',
        'overflow-y': 'auto'  # To ensure a vertical scroll
    }),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'993149cf-59df-4350-8d7d-34dd4d15f65d': {'version…

# **Lab Report: M-Shots **

**Exploring M-Shots Learning: Applications in Chatbots**

**Introduction**

This report explores the application of M-Shots Learning techniques in building chatbots specialized in Formula 1 (F1), FIFA & International Football Competitions, and Movies. Techniques such as Zero-Shot, One-Shot, and Few-Shots Learning were employed to evaluate the chatbot’s ability to provide accurate and context-aware responses. The experiments aimed to understand the impact of prompt engineering on improving chatbot performance and addressing complex queries.

**Experiments and Findings**

### F1 Chatbot

- **What Worked:**
  - Few-Shots Learning significantly improved response accuracy and formatting. Examples provided within the context ensured consistency.
  - Accurate responses to direct and well-structured questions.
  - Flexibility in handling ambiguous queries with some degree of context.
- **What Didn't Work:**
  - Inconsistent formatting when creative prompts were used, often leading to incomplete or hallucinated responses.
  - Difficulty maintaining strict response formats when multiple data points were requested.
- **Key Learning:** Structured examples were critical for improving reliability, and the chatbot required explicit instructions to avoid hallucinations.

### FIFA Chatbot

- **What Worked:**
  - Few-Shots Learning enhanced the chatbot’s ability to answer detailed questions about past FIFA competitions and players.
  - Incorporation of a disclaimer for questions beyond the September 2021 cutoff improved user transparency.
  - Consistent responses for data within the model's knowledge range.
- **What Didn't Work:**
  - The chatbot occasionally failed to follow strict formats, especially under ambiguous or compound questions.
  - Struggled with nuanced queries about competitions that did not occur or were incomplete, requiring additional context.
- **Key Learning:** Few-Shots Learning guided the chatbot effectively, but validation mechanisms were required to enforce format consistency.

### Movie Chatbot

- **What Worked:**
  - Accurate answers for movies released before 2021, especially when structured prompts and examples were used.
  - The chatbot provided detailed data, including awards, cast, and ratings, as per the desired response format.
  - Flexibility to answer both actor-specific and movie-specific questions.
- **What Didn't Work:**
  - Failed to adhere to the strict format consistently, especially when generating long responses.
  - Responses for ambiguous queries were less precise and occasionally omitted required fields.
- **Key Learning:** While Few-Shots Learning provided a strong baseline, the chatbot’s tendency to deviate from the specified format highlighted the need for stricter validation.

**Comparative Analysis**

- **Response Accuracy:** The F1 chatbot demonstrated the highest accuracy due to the simplicity of data points (e.g., championships and drivers). The FIFA chatbot performed well but struggled with ambiguous queries, while the Movie chatbot faced challenges in adhering to strict formats due to the diversity of data points.
- **Flexibility:** The Movie chatbot exhibited the most flexibility, handling a wide range of queries about awards, directors, and actors. However, this flexibility sometimes led to errors.
- **Consistency:** Few-Shots Learning improved consistency across all chatbots, but the F1 chatbot showed the most consistent adherence to the response format.
- **Error Mitigation:** Adding disclaimers in the FIFA chatbot enhanced user understanding of the model’s limitations, which was a valuable addition.

**Conclusion**

M-Shots Learning techniques proved to be a powerful tool for enhancing chatbot performance. Few-Shots Learning consistently delivered more structured and accurate responses, demonstrating the importance of examples in guiding the model. However, validation mechanisms are necessary to ensure strict adherence to predefined response formats, especially for complex or diverse data points.

**Key Takeaways:**
1. Structured examples and explicit guidelines are critical for improving response accuracy and consistency.
2. Validation mechanisms should be incorporated to handle ambiguous or compound queries effectively.
3. Adding disclaimers for knowledge cutoffs increases user transparency and trust.
4. Flexibility in handling creative prompts must be balanced with strict format adherence to minimize hallucinations.

Future work should focus on dynamic context updates, token optimization, and integrating validation layers to ensure higher accuracy and reliability across diverse chatbot applications.